`conda activate imgpro`

In [2]:
import sys
import os
import glob
import importlib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import utils
utils.default_plt_params()

import grid_analysis

In [3]:
DATA_PTH = r'D:\DATA'
PROJECT = 'g5ht-free' # g5ht-free or g5ht-immo
OUT_PTH = 'date-20260123_strain-ISg5HT-nsIS180_condition-fedpatch_worm004'
reg_dir = 'registered_elastix'
# reg_dir = 'registered_wholistic_smooth-0.200_patch-7' # for 20251223 worm005

DATE = OUT_PTH.split('_')[0].split('-')[1] 
DATA_PTH = os.path.join(DATA_PTH, PROJECT, DATE)
pth = os.path.join(DATA_PTH, OUT_PTH)

reg_dir = os.path.join(pth, reg_dir)
reg_tifs = [f for f in os.listdir(reg_dir) if f.endswith('.tif')]
reg_tifs.sort() # ensure correct order
reg_tifs_pths = [os.path.join(reg_dir, f) for f in reg_tifs]

# load mask
mask_fn = glob.glob(os.path.join(pth, 'fixed_mask_[0-9][0-9][0-9][0-9]*.tif'))[0]
mask = tifffile.TiffFile(os.path.join(pth, mask_fn)).asarray()
print(f"Mask shape: {mask.shape}")

# load first frame to get dimensions
with tifffile.TiffFile(reg_tifs_pths[0]) as tif:
    Z,C,H,W = tif.asarray().shape
print(f"Dimensions of registered tifs: Z={Z}, C={C}, H={H}, W={W}")


Mask shape: (200, 500)
Dimensions of registered tifs: Z=39, C=2, H=200, W=500


In [5]:
xspace = 20
yspace = 50

grid_info, n_grids_y, n_grids_x = grid_analysis.create_grid(H,W,xspace,yspace) # (grid_info = (grid_id, y_start, y_end, x_start, x_end))
grid_intensity = np.full((len(reg_tifs_pths), n_grids_y, n_grids_x), np.nan) # shape (frames, n_grids_y, n_grids_x)
frame_indices = np.array([os.path.basename(p).split('.')[0] for p in reg_tifs_pths], dtype=int)

In [149]:
# grid up the volume, get mean across each grid square and , and plot the resulting grid

xspace = 20
yspace = 50

grid_info, n_grids_y, n_grids_x = grid_analysis.create_grid(H,W,xspace,yspace) # (grid_info = (grid_id, y_start, y_end, x_start, x_end))
grid_intensity = np.full((len(reg_tifs_pths), n_grids_y, n_grids_x), np.nan) # shape (frames, n_grids_y, n_grids_x)
frame_indices = np.array([os.path.basename(p).split('.')[0] for p in reg_tifs_pths], dtype=int)

for iframe,reg_tif_pth in tqdm(enumerate(reg_tifs_pths)):
    with tifffile.TiffFile(reg_tif_pth) as tif:
        frame = tif.asarray() # shape (C,Z,H,W)

    # calculate ratiometric intensity for each grid square
    for iy in range(n_grids_y):
        for ix in range(n_grids_x):
            y_start, y_end, x_start, x_end = grid_info[iy * n_grids_x + ix][1:] # get grid coordinates
            
            grid_mask = mask[y_start:y_end, x_start:x_end]
            grid_data = frame[:, :, y_start:y_end, x_start:x_end]
            grid_means = np.mean(grid_data, axis=(2,3)) # mean across Z,H,W dimensions (shape (C,Z))
            grid_max = np.max(grid_means, axis=0) # max across Z dimension (shape (C,))
            if grid_max[1] == 0: # avoid division by zero
                grid_ratio = np.nan
            else:
                grid_ratio = grid_max[0] / grid_max[1] # ratiometric gfp/rfp
            grid_intensity[iframe, iy, ix] = grid_ratio


1108it [00:48, 22.99it/s]


In [152]:
grid_intensity_flat = grid_intensity.reshape(len(reg_tifs_pths), -1) # shape (frames, n_grids_y*n_grids_x)

baseline_window = (201, 240) # frames, this is your intial guess, adjust based on plots below

In [153]:
%matplotlib qt
plt.close('all')

fig, ax = utils.pretty_plot(figsize=(10,6))
ax.plot(grid_intensity_flat, lw=0.1)
ax.plot(np.nanmean(grid_intensity_flat, axis=1), color='k', lw=2)
ax.set_xlabel('Frame')
ax.set_ylabel('Mean GFP/RFP Ratio')
plt.show()

# plot as heatmap
fig, ax = utils.pretty_plot(figsize=(10,6))
im = ax.pcolormesh(grid_intensity_flat.T, shading='auto', cmap='viridis', vmin=np.nanpercentile(grid_intensity_flat, 0), vmax=np.nanpercentile(grid_intensity_flat, 90))
ax.set_xlabel('Frame')
ax.set_ylabel('Grid Square')
fig.colorbar(im, ax=ax, label='Mean GFP/RFP Ratio')
plt.show()

# same plots but intensity divided by baseline window
baseline_mean = np.nanmean(grid_intensity_flat[baseline_window[0]:baseline_window[1], :], axis=0) # shape (n_grids_y*n_grids_x,)
grid_intensity_normalized = grid_intensity_flat / baseline_mean # shape (frames, n_grids_y*n_grids_x)

# # calculate baseline as F/F20, where F20 is the 20th percentile of each grid square across the entire time series
# baseline_f20 = np.nanpercentile(grid_intensity_flat, 20, axis=0) # shape (n_grids_y*n_grids_x,)
# grid_intensity_normalized = grid_intensity_flat / baseline_f20 # shape (frames, n_grids_y*n_grids_x

fig, ax = utils.pretty_plot(figsize=(10,6))
ax.plot(grid_intensity_normalized, lw=0.1)
ax.plot(np.nanmean(grid_intensity_normalized, axis=1), color='k', lw=2)
ax.set_xlabel('Frame')
ax.set_ylabel('Mean GFP/RFP Ratio')
plt.show()

# plot as heatmap
fig, ax = utils.pretty_plot(figsize=(10,6))
im = ax.pcolormesh(grid_intensity_normalized.T, shading='auto', cmap='viridis', vmin=np.nanpercentile(grid_intensity_normalized, 0), vmax=np.nanpercentile(grid_intensity_normalized, 99))
ax.set_xlabel('Frame')
ax.set_ylabel('Grid Square')
fig.colorbar(im, ax=ax, label='Mean GFP/RFP Ratio')
plt.show()


In [7]:
import json

# metadata
# NOTE: all frames are indexed from 0 to nframes-1, so frame_index[0] corresponds to frame 0, frame_index[1] to frame 1, etc. 
# This means that if you want to specify bad_frames, baseline_start_frame, baseline_end_frame, or encounter_frame, 
# you should use the indices corresponding to the frames in the registered tifs, 
# not the absolute frame numbers from the original acquisition (unless they happen to be the same). 
# For example, if your first registered tif corresponds to frame 100 in the original acquisition, 
# then frame_index[0] would be 100, and if you want to mark that as a bad frame, you would include 0 in bad_frames, not 100.
fps = (1/0.533) # Hz
nframes = len(reg_tifs_pths)
frame_index = frame_indices
baseline_start_frame = 490 # int, set to None if you don't have a clear baseline period, or if you want to use the entire time series to calculate baseline statistics like F20
baseline_end_frame = 540 # int, set to None if you don't have a clear baseline period, or if you want to use the entire time series to calculate baseline statistics like F20
encounter_frame = 675 # int, set to None if no encounter
# bad_frames = np.array([])
bad_frames = np.arange(0, 487)
# bad_frames1 = np.arange(0,200)
# bad_frames2 = np.arange(339,356)
# bad_frames = np.concatenate([bad_frames1, bad_frames2]) 
# bad_frames frame indices to exclude (indexed into range(0, nframes))
# # bad_frames are in range(0,nframes), not absolute frame numbers, so they can be used to index into range(nframes) or frame_index

metadata = {
    'fps': fps,
    'nframes': nframes,
    'baseline_start_frame': baseline_start_frame,
    'baseline_end_frame': baseline_end_frame,
    'encounter_frame': encounter_frame,
    'bad_frames': bad_frames.tolist(), # convert numpy array to list for JSON
    'frame_index': frame_index.tolist(),  # convert numpy array to list for JSON
}

meta_pth = os.path.join(pth, 'metadata.json')
with open(meta_pth, 'w') as f:
    json.dump(metadata, f, indent=4)
print(f"Metadata saved to {meta_pth}")

Metadata saved to D:\DATA\g5ht-free\20260123\date-20260123_strain-ISg5HT-nsIS180_condition-fedpatch_worm004\metadata.json


In [ ]:
# --- Loading metadata in a subsequent analysis ---
with open(meta_pth, 'r') as f:
    metadata = json.load(f)
metadata['bad_frames'] = np.array(metadata['bad_frames'])  # convert back to numpy array
metadata['frame_index'] = np.array(metadata['frame_index'])  # convert back to numpy array
print(metadata)

In [1]:
# copy all metadata files to a new directory
# when copying, rename them from metadata.json to metadata_{OUT_PTH}.json

import os
import json

dest_folder = r'D:\DATA\g5ht-free-metadata'

# loop over all recordings
DATA_PTH = r'D:\DATA\g5ht-free'
worm_tuple = ('worm001', 'worm002', 'worm003', 'worm004', 'worm005', 'worm006', 'worm007', 'worm008', 'worm009', 'worm010', 'worm011')

date_pths = [os.path.join(DATA_PTH, d) for d in os.listdir(DATA_PTH) if os.path.isdir(os.path.join(DATA_PTH, d))]

# in each folder in date_pths, look for folders that end with 'wormXXX', where XXX is a three digit number, and then look for the 'registered_elastix' folder inside that folder to create the mp4
for date_pth in date_pths:
    print(date_pth)
    worm_pths = [os.path.join(date_pth, d) for d in os.listdir(date_pth) if os.path.isdir(os.path.join(date_pth, d)) and d.endswith(worm_tuple)]
    print(worm_pths)
    for worm_pth in worm_pths:
        meta_pth = os.path.join(worm_pth, 'metadata.json')
        if os.path.exists(meta_pth):
            with open(meta_pth, 'r') as f:
                metadata = json.load(f)
            out_meta_pth = os.path.join(dest_folder, f'metadata_{os.path.basename(worm_pth)}.json')
            with open(out_meta_pth, 'w') as f:
                json.dump(metadata, f, indent=4)
            print(f"Copied metadata from {meta_pth} to {out_meta_pth}")


D:\DATA\g5ht-free\022025_eft_41z_starved_worm002
['D:\\DATA\\g5ht-free\\022025_eft_41z_starved_worm002\\022025_eft_41z_starved_worm002']
D:\DATA\g5ht-free\20251223
['D:\\DATA\\g5ht-free\\20251223\\date-20251223_strain-ISg5HT_condition-starvedpatch_worm005', 'D:\\DATA\\g5ht-free\\20251223\\date-20251223_strain-ISg5HT_condition-fedpatch_worm001', 'D:\\DATA\\g5ht-free\\20251223\\date-20251223_strain-ISg5HT_condition-starvedpatch_worm004']
Copied metadata from D:\DATA\g5ht-free\20251223\date-20251223_strain-ISg5HT_condition-starvedpatch_worm005\metadata.json to D:\DATA\g5ht-free-metadata\metadata_date-20251223_strain-ISg5HT_condition-starvedpatch_worm005.json
Copied metadata from D:\DATA\g5ht-free\20251223\date-20251223_strain-ISg5HT_condition-starvedpatch_worm004\metadata.json to D:\DATA\g5ht-free-metadata\metadata_date-20251223_strain-ISg5HT_condition-starvedpatch_worm004.json
D:\DATA\g5ht-free\20260123
['D:\\DATA\\g5ht-free\\20260123\\date-20260123_strain-ISg5HT-nsIS180_condition-fedpat